**Start the Knowledge graph vectorDB in neo4j Desktop**

In [ ]:
import dotenv
from dotenv import load_dotenv
import os
from groq import Groq
# from langchain.schema import SystemMessage, HumanMessage
from langchain_community.graphs import Neo4jGraph
load_dotenv()
# Warning control
import warnings
warnings.filterwarnings("ignore")

**Load the LLM**

In [ ]:
api_key = "Replace with your actual Groq API key"  
client = Groq(api_key=api_key)

**Add Neo4j credentials (These information need to be kept secret)**

In [ ]:
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "Enter user ID"
NEO4J_PASSWORD = "Enter password"
NEO4J_DATABASE = 'neo4j'

In [ ]:

graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)

**Print the graph database schema**

In [ ]:
graph.refresh_schema()
print(graph.schema)

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

groq_api_key="Replace with your actual Groq API key" 

In [ ]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-70b-versatile")
llm


In [ ]:
!pip install --upgrade --quiet langchain_experimental
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

Questions

### **Chain**

**`Simple Agent (a)`:**

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,  # Acknowledge the risks
    validate_cypher=True            # Enable Cypher validation
)

print(chain)


In [ ]:


q_one = "I want to recycle lithium ion batteries with at least 97.5% Lithium (Li) recovery. What processes can achieve this?"
q_two = "I want to recycle Lithium-ion batteries with 100% Lithium (Li) recovery. What processes can achieve this?"
q_three = "I want to recycle Lithium-ion batteries with 95% Nickel (Ni) recovery. Which processes can achieve this in minimum cost?"

response = chain.invoke({"query": q_one})
# Print intermediate Cypher query for debugging
print("Generated Cypher Query:", response.get("intermediate_steps", {}).get("cypher_query", "No query generated"))
# Handle response and errors
if "result" in response:
    print("\nLLM response:", response["result"])
else:
    print("\nError in response:", response)

#print(response)
#print("\nLLM response:", response["result"])

### **`Special Agent`**

In [ ]:
import re
!pip install rapidfuzz
from rapidfuzz import process

q_one = "I want to recycle lithium ion batteries with at least 95% Rare Earth Elements recovery. Which processes can achieve this?"
q_two = "I want to recycle lithium ion batteries. And I want processes which recovers Indium element as it's expensive material. What processes should I use?"
q_three = "Can you suggest battery recycling processes which have 100% Cobalt (Co) recovery rate along with the conditions?"

from langchain.chains import GraphCypherQAChain

# Function to parse the user query and extract thresholds dynamically

# List of possible materials
material_list = [
    "Nickel (Ni)", "Lithium (Li)", "Cobalt (Co)", "Copper (Cu)", "Zinc (Zn)", 
    "Iron (Fe)", "Graphite (C (Graphite))", "Indium (In)", "Manganese (Mn)", 
    "Aluminium (Al)", "Cadmium (Cd)", "Phosphorus (P)", "Rare Earth Elements (REEs)"
]

# Function to extract and normalize material using fuzzy matching
def extract_material_with_fuzzy(query):
    # Clean the query (to handle abbreviations and case sensitivity)
    query_normalized = query.lower()
    # Match the query against the material list using fuzzy matching
    matched_material, score, index = process.extractOne(query_normalized, material_list)
    # You can optionally use 'score' or 'index' if needed, or just return the matched material
    return matched_material

# Updated extract_threshold_and_material function
def extract_threshold_and_material(query):
    # Extract the numeric threshold (e.g., "99.9%")
    threshold_match = re.search(r"([\d.]+)\s*%?", query, re.IGNORECASE)
    threshold = float(threshold_match.group(1)) if threshold_match else 0.0

    # Use fuzzy matching to extract the material of interest
    material = extract_material_with_fuzzy(query)
    
    return threshold, material


# Function to validate and adjust generated Cypher queries dynamically
def validate_cypher_query(cypher_query, threshold, material):
    # Dynamically replace the material in the Cypher query
    fixed_query = f"""
    MATCH (bt:Battery_Type)-[:Processed_By]->(rpu:Recycling_Process_Used)-[rel:Recovers]->(rm:Recovered_Materials)
    MATCH (rpu)-[:Has_Conditions]->(c:Process_Conditions)
    WHERE toFloat(rel.efficiency) >= {threshold} AND rm.name = "{material}"
    RETURN DISTINCT 
    rpu.name AS Process, 
    rpu.total_cost AS Cost, 
    rpu.total_energy AS Energy, 
    rel.efficiency AS Efficiency,
    c.details AS Conditions
    """
    return fixed_query

# Extract threshold and material from the user query
threshold, material = extract_threshold_and_material(q_three)

response = chain.invoke({"query": q_three})

# Debugging: Validate the generated Cypher query
generated_query = response.get("intermediate_steps", {}).get("cypher_query", "")
print("Generated Cypher Query (Before Validation):", generated_query)

# Validate and fix the query
fixed_query = validate_cypher_query(generated_query, threshold, material)
print("Fixed Cypher Query:", fixed_query)

# Execute the fixed query directly
if fixed_query:
    context = graph.query(fixed_query)
    print("Query Results:", context)
else:
    print("\nError: No query generated or fixed query is empty.")



In [ ]:
from langchain.chains import GraphCypherQAChain

# Function to format query results into context
def format_query_results(results):
    if results:
        formatted_context = "The following recycling processes meet the specified criteria:\n\n"
        for result in results:
            process = result.get("Process", "N/A")
            efficiency = result.get("Efficiency", "N/A")
            Conditions = result.get("Conditions", "N/A")
            cost = result.get("Cost", "N/A")
            energy = result.get("Energy", "N/A")

            formatted_context += (
                f"  - Process: {process}\n"
                f"  - Efficiency: {efficiency}%\n"
                f"  - Conditions: {Conditions}\n"
                f"  - Total Cost: ${cost}\n"
                f"  - Total Energy: {energy} J\n\n"
            )
    else:
        formatted_context = "No processes meet the specified criteria."
    
    return formatted_context

# Format query results
formatted_context = format_query_results(context)
print("Formatted Context:", formatted_context)



In [ ]:
response = chain.invoke(formatted_context)
print(response)  # Output: "I'd be happy to help with that query."

In [ ]:
response = chain.invoke({"query": q_two})
print(response)
print("\nLLM response:", response["result"])

In [ ]:
q_three = "I want to recycle batteries in minimum cost, so which process should I use? And what are the conditions for that process"

In [ ]:
response = chain.invoke({"query": q_three})
print(response)
print("\nLLM response:", response["result"])

In [ ]:
q_four = "I want to recycle batteries in minimum energy requirement, so which process should I use? And what are the conditions for that process"

In [ ]:
response = chain.invoke({"query": q_four})
print(response)
print("\nLLM response:", response["result"])

In [ ]:
q_five = "I want to recycle batteries. And I want processes which recovers Indium (In) element as it's an expensive material. What processes & conditions should I use?"

In [ ]:
response = chain.invoke({"query": q_five})
print(response)
print("\nLLM response:", response["result"])

In [ ]:
q_six = "I want to recycle batteries. And I want processes which can recover Lithium (Li), Cobalt (Co) & Manganeses (Mn) with minimum cost. What processes should I use & it's recovery rate with process conditions?"

In [ ]:
response = chain.invoke({"query": q_six})
print(response)
print("\nLLM response:", response["result"])

In [ ]:
q_seven = "Can you suggest battery recycling processes which recovers Cobalt (Co) & Lithium (Li) which has minimum energy requirement along with the process conditions & recovery rates?"

In [ ]:
response = chain.invoke({"query": q_seven})
print(response)
print("\nLLM response:", response["result"])